In [1]:
# 🔬 These Three Correspond To Three Research Questions
# Method	Tests
# First-K	positional bias
# Uniform	spatial sparsity
# Mean Pool	spatial resolution importance

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoProcessor, AutoModel, AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import itertools

device = "cuda"
torch.set_float32_matmul_precision("high")

In [3]:
# =====================================================
# SEARCH SPACE
# =====================================================

TOKEN_COUNTS = [196, 64, 32, 16, 8]
REDUCTION_METHODS = ["uniform", "meanpool"]
PROJECTOR_TYPES = ["linear", "mlp"]
LEARNING_RATES = [1e-4, 5e-5]

TRAIN_STEPS = 800

In [ ]:
# =====================================================
# 4️⃣ Load 5 Images + Captions
# =====================================================

data = [
    ("images/airplane.png",
     "A large passenger airplane flying through the air."),

    ("images/motorcycle.png",
     "Riding a motorcycle down a street."),

    ("images/gd-dog.jpg",
     "A dog standing in a grassy field."),

    ("images/kitchen.png",
     "A kitchen stove, sink, and counter with stuff on it."),

    ("images/person-umbrella.png",
     "A person walking in the rain while holding an umbrella.")
]

In [5]:
# =====================================================
# Token Reduction
# =====================================================

def reduce_tokens(patch, method, K):

    if K >= patch.size(1):
        return patch

    if method == "uniform":
        indices = torch.linspace(
            0, patch.size(1)-1, K
        ).long().to(patch.device)
        return patch[:, indices, :]

    elif method == "meanpool":
        N = patch.size(1)
        chunk = N // K
        pooled = []
        for i in range(K):
            pooled.append(
                patch[:, i*chunk:(i+1)*chunk, :].mean(dim=1, keepdim=True)
            )
        return torch.cat(pooled, dim=1)

In [6]:
# =====================================================
# Vision Influence Metric
# =====================================================

def vision_influence(llm, patch_tokens):

    with torch.no_grad():
        random_visual = torch.randn_like(patch_tokens) * 0.01

        logits_v = llm(inputs_embeds=patch_tokens).logits[:, -1, :]
        logits_r = llm(inputs_embeds=random_visual).logits[:, -1, :]

        diff = torch.norm(logits_v - logits_r).item()
        base = torch.norm(logits_r).item()

    return diff / (base + 1e-8)

In [ ]:
# =====================================================
# Run Single Experiment
# =====================================================

def run_experiment(token_count, method, projector_type, lr):

    print(f"\nRunning: tokens={token_count}, method={method}, "
          f"proj={projector_type}, lr={lr}")

    # Load models fresh each run
    vision_name = "google/siglip-base-patch16-224"
    processor = AutoProcessor.from_pretrained(vision_name, use_fast=True)

    vision_model = AutoModel.from_pretrained(
        vision_name
    ).vision_model.to(device, dtype=torch.bfloat16)

    vision_model.eval()
    for p in vision_model.parameters():
        p.requires_grad = False

    llm_name = "Qwen/Qwen3-0.6B"
    tokenizer = AutoTokenizer.from_pretrained(llm_name)
    tokenizer.pad_token = tokenizer.eos_token

    llm = AutoModelForCausalLM.from_pretrained(
        llm_name,
        dtype=torch.bfloat16
    ).to(device)

    hidden = llm.config.hidden_size

    # Projector
    if projector_type == "linear":
        projector = nn.Linear(768, hidden, bias=False)

    else:
        projector = nn.Sequential(
            nn.Linear(768, hidden),
            nn.GELU(),
            nn.Linear(hidden, hidden)
        )

    projector = projector.to(device, dtype=torch.bfloat16)

    optimizer = torch.optim.AdamW(
        list(projector.parameters()) + list(llm.parameters()),
        lr=lr
    )

    # Precompute patches
    dataset = []

    for img_path, caption in data:
        image = Image.open(img_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = vision_model(**inputs)
            patch = outputs.last_hidden_state.detach()

        patch = reduce_tokens(patch, method, token_count)

        tokens = tokenizer(caption, return_tensors="pt").to(device)
        dataset.append((patch, tokens, caption))

    # Training
    llm.train()
    projector.train()

    for step in range(0, TRAIN_STEPS+1):

        total_loss = 0

        for patch_raw, tokens, _ in dataset:

            patch_tokens = projector(patch_raw)

            text_embeds = llm.get_input_embeddings()(tokens.input_ids)

            inputs_embeds = torch.cat([patch_tokens, text_embeds], dim=1)

            visual_attention = torch.ones(
                (1, patch_tokens.size(1)),
                device=device,
                dtype=tokens.attention_mask.dtype
            )

            full_attention = torch.cat(
                [visual_attention, tokens.attention_mask],
                dim=1
            )

            visual_label_pad = torch.full(
                (1, patch_tokens.size(1)),
                -100,
                device=device
            )

            full_labels = torch.cat(
                [visual_label_pad, tokens.input_ids],
                dim=1
            )

            outputs = llm(
                inputs_embeds=inputs_embeds,
                attention_mask=full_attention,
                labels=full_labels
            )

            loss = outputs.loss
            loss.backward()
            total_loss += loss.item()

        optimizer.step()
        optimizer.zero_grad()

    final_loss = total_loss / len(dataset)

    # Evaluation accuracy
    llm.eval()
    projector.eval()

    correct = 0

    for patch_raw, _, caption in dataset:

        with torch.no_grad():
            patch_tokens = projector(patch_raw)

            generated = llm.generate(
                inputs_embeds=patch_tokens,
                attention_mask=torch.ones(
                    (1, patch_tokens.size(1)),
                    device=device
                ),
                max_new_tokens=15,
                do_sample=False
            )

        output = tokenizer.decode(generated[0], skip_special_tokens=True)

        if caption in output:
            correct += 1

    accuracy = correct / len(dataset)

    influence = vision_influence(llm, projector(dataset[0][0]))

    # Cleanup
    del llm
    del projector
    del vision_model
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    return {
        "tokens": token_count,
        "method": method,
        "projector": projector_type,
        "lr": lr,
        "loss": final_loss,
        "accuracy": accuracy,
        "influence": influence
    }


In [8]:
# =====================================================
# MAIN SEARCH LOOP
# =====================================================

results = []

for config in itertools.product(
    TOKEN_COUNTS,
    REDUCTION_METHODS,
    PROJECTOR_TYPES,
    LEARNING_RATES
):
    results.append(run_experiment(*config))

print("\n\n===== FINAL RESULTS =====")
for r in sorted(results, key=lambda x: (-x["accuracy"], x["loss"])):
    print(r)



Running: tokens=196, method=uniform, proj=linear, lr=0.0001


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Running: tokens=196, method=uniform, proj=linear, lr=5e-05

Running: tokens=196, method=uniform, proj=mlp, lr=0.0001

Running: tokens=196, method=uniform, proj=mlp, lr=5e-05

Running: tokens=196, method=meanpool, proj=linear, lr=0.0001

Running: tokens=196, method=meanpool, proj=linear, lr=5e-05

Running: tokens=196, method=meanpool, proj=mlp, lr=0.0001

Running: tokens=196, method=meanpool, proj=mlp, lr=5e-05

Running: tokens=64, method=uniform, proj=linear, lr=0.0001

Running: tokens=64, method=uniform, proj=linear, lr=5e-05

Running: tokens=64, method=uniform, proj=mlp, lr=0.0001

Running: tokens=64, method=uniform, proj=mlp, lr=5e-05

Running: tokens=64, method=meanpool, proj=linear, lr=0.0001

Running: tokens=64, method=meanpool, proj=linear, lr=5e-05

Running: tokens=64, method=meanpool, proj=mlp, lr=0.0001

Running: tokens=64, method=meanpool, proj=mlp, lr=5e-05

Running: tokens=32, method=uniform, proj=linear, lr=0.0001

Running: tokens=32, method=uniform, proj=linear, lr=5e-0